# Ejercicio - Busqueda de Alojamiento en Airbnb.

Supongamos que somos un agente de [Airbnb](http://www.airbnb.com) localizado en Lisboa, y tenemos que atender peticiones de varios clientes. Tenemos un archivo llamado `airbnb.csv` (en la carpeta data) donde tenemos información de todos los alojamientos de Airbnb en Lisboa.

In [1]:
import pandas as pd
df_airbnb = pd.read_csv("./src/airbnb.csv")

In [2]:
df_airbnb.head()

,room_id,host_id,room_type,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price
0,6499,14455,Entire home/apt,Belém,8,5.0,2,1.0,57.0
1,17031,66015,Entire home/apt,Alvalade,0,0.0,2,1.0,46.0
2,25659,107347,Entire home/apt,Santa Maria Maior,63,5.0,3,1.0,69.0
3,29248,125768,Entire home/apt,Santa Maria Maior,225,4.5,4,1.0,58.0
4,29396,126415,Entire home/apt,Santa Maria Maior,132,5.0,4,1.0,67.0


In [3]:
df_airbnb.dtypes

room_id                   int64
host_id                   int64
room_type                object
neighborhood             object
reviews                   int64
overall_satisfaction    float64
accommodates              int64
bedrooms                float64
price                   float64
dtype: object

En concreto el dataset tiene las siguientes variables:
- room_id: el identificador de la propiedad
- host_id: el identificador del dueño de la propiedad
- room_type: tipo de propiedad (vivienda completa/(habitacion para compartir/habitación privada)
- neighborhood: el barrio de Lisboa
- reviews: El numero de opiniones
- overall_satisfaction: Puntuacion media del apartamento
- accommodates: El numero de personas que se pueden alojar en la propiedad
- bedrooms: El número de habitaciones
- price: El precio (en euros) por noche

## Usando Pandas

### Caso 1.

Alicia va a ir a Lisboa durante una semana con su marido y sus 2 hijos. Están buscando un apartamento con habitaciones separadas para los padres y los hijos. No les importa donde alojarse o el precio, simplemente quieren tener una experiencia agradable. Esto significa que solo aceptan lugares con más de 10 críticas con una puntuación mayor de 4. Cuando seleccionemos habitaciones para Alicia, tenemos que asegurarnos de ordenar las habitaciones de mejor a peor puntuación. Para aquellas habitaciones que tienen la misma puntuación, debemos mostrar antes aquellas con más críticas. Debemos darle 3 alternativas.

In [5]:
# Consejo: para mostrar 3 alternativas emplee head(3)

# condiciones del problema
condition = ((df_airbnb['reviews'] >10) & (df_airbnb['overall_satisfaction'] >4))
condition_2 = ((df_airbnb['accommodates']==4) & (df_airbnb['bedrooms']==3))

df_filter = df_airbnb[condition & condition_2]

# ordenamiento
df_order = df_filter.sort_values(['overall_satisfaction', 'reviews'], ascending=[False, False])

# top 3 
df_top3 = df_order.head(3)
print(df_top3)

      room_id   host_id        room_type       neighborhood  reviews  \
1865  2777355  14204351  Entire home/apt            Estrela       89   
4100  6408578  33201646  Entire home/apt       Misericórdia       86   
1153  1374173   2635324  Entire home/apt  Santa Maria Maior       56   

      overall_satisfaction  accommodates  bedrooms  price  
1865                   5.0             4       3.0   58.0  
4100                   5.0             4       3.0   75.0  
1153                   5.0             4       3.0   93.0  


### Caso 2

Roberto es un casero que tiene una casa en Airbnb. De vez en cuando nos llama preguntando sobre cuales son las críticas de su alojamiento. Hoy está particularmente enfadado, ya que su hermana Clara ha puesto una casa en Airbnb y Roberto quiere asegurarse de que su casa tiene más críticas que las de Clara. Tenemos que crear un dataframe con las propiedades de ambos. Las id de las casas de Roberto y Clara son 97503 y 90387  respectivamente. Finalmente guardamos este dataframe como excel llamado "roberto.xls

In [11]:
import pandas as pd

# Identificadores de las casas de Roberto y Clara
id_roberto = 97503
id_clara = 90387

# Verificar que los host_id estén presentes en el conjunto de datos original
if id_roberto not in df_airbnb['host_id'].values:
    print(f"El host_id {id_roberto} de Roberto no está en el conjunto de datos.")
if id_clara not in df_airbnb['host_id'].values:
    print(f"El host_id {id_clara} de Clara no está en el conjunto de datos.")

# Filtrar propiedades de Roberto y Clara
df_roberto = df_airbnb[df_airbnb['host_id'] == id_roberto]
df_clara = df_airbnb[df_airbnb['host_id'] == id_clara]

# Imprimir los DataFrames individuales
print("DataFrame de Roberto:")
print(df_roberto)

print("\nDataFrame de Clara:")
print(df_clara)

# Concatenar los DataFrames de Roberto y Clara
df_roberto_clara = pd.concat([df_roberto, df_clara])

# Guardar el DataFrame en un archivo Excel
df_roberto_clara.to_excel("roberto.xlsx", index=False)

# Imprimir el DataFrame combinado
print("\nDataFrame combinado:")
print(df_roberto_clara)

El host_id 97503 de Roberto no está en el conjunto de datos.
El host_id 90387 de Clara no está en el conjunto de datos.
DataFrame de Roberto:
Empty DataFrame
Columns: [room_id, host_id, room_type, neighborhood, reviews, overall_satisfaction, accommodates, bedrooms, price]
Index: []

DataFrame de Clara:
Empty DataFrame
Columns: [room_id, host_id, room_type, neighborhood, reviews, overall_satisfaction, accommodates, bedrooms, price]
Index: []

DataFrame combinado:
Empty DataFrame
Columns: [room_id, host_id, room_type, neighborhood, reviews, overall_satisfaction, accommodates, bedrooms, price]
Index: []



### Caso 3

Diana va a Lisboa a pasar 3 noches y quiere conocer a gente nueva. Tiene un presupuesto de 50€ para su alojamiento. Debemos buscarle las 10 propiedades más baratas, dandole preferencia a aquellas que sean habitaciones compartidas *(room_type == Shared room)*, y para aquellas viviendas compartidas debemos elegir aquellas con mejor puntuación.

In [12]:
# Condiciones del problema para Diana
condition_diana = ((df_airbnb['price'] <= 50) & (df_airbnb['room_type'] == 'Shared room'))

# Filtrar propiedades para Diana
df_diana = df_airbnb[condition_diana]

# Ordenar por precio ascendente y luego por puntuación descendente para las habitaciones compartidas
df_diana = df_diana.sort_values(['price', 'overall_satisfaction'], ascending=[True, False])

# Obtener las 10 propiedades más baratas
df_diana_top10 = df_diana.head(10)

# Imprimir las 10 propiedades más baratas para Diana
print(df_diana_top10)

        room_id    host_id    room_type       neighborhood  reviews  \
1010    1179457    5799522  Shared room      Santo António       42   
3562    5557699   28812904  Shared room  Santa Maria Maior       22   
7584   13116032   72951043  Shared room            Arroios        1   
13148  19314160  135270245  Shared room        Santa Clara        0   
9065   14933182   91501272  Shared room      Santo António        5   
4353    6728244   28812904  Shared room  Santa Maria Maior        8   
5616    9317561   48360716  Shared room            Arroios       13   
6640   11693356   28812904  Shared room  Santa Maria Maior        3   
6641   11693442   28812904  Shared room  Santa Maria Maior       16   
8908   14708916   91501272  Shared room      Santo António       18   

       overall_satisfaction  accommodates  bedrooms  price  
1010                    4.0            16       1.0   10.0  
3562                    4.0             1       1.0   10.0  
7584                    0.0        